In [ ]:
import os
import pandas as pd
import pickle


%config Completer.use_jedi = False

joinpath = os.path.join

os.getcwd()

In [ ]:
rawdatainfo = pd.read_csv('datamanager/overview_rawdata.csv', sep=';')
rawdatainfo

In [ ]:
file = 'data/computed_data/city_codes.pickle'
with open(file, 'rb') as f:
    city_codes = pickle.load(f)
print(city_codes.isna().sum())

city_codes

In [ ]:
raw_data_path = 'data/raw_data/'
dimensions_path = os.path.join(raw_data_path, 'dimensions')

!ls {raw_data_path}

In [ ]:
!head {os.path.join(dimensions_path,'Index.csv ') }

In [ ]:
for file in os.listdir(dimensions_path): 
    print('\n', file)
    !head {os.path.join(dimensions_path, file) }

In [ ]:
rawdatainfo = pd.read_csv(joinpath('datamanager/overview_rawdata.csv'))

In [ ]:
citytemp = pd.read_csv('data/raw_data/dimensions/city_temperature_2020.csv')
citytemp.rename(columns={'City': 'city'}, inplace=True)
res = citytemp.merge(city_codes, how='left', on='city')

#if res.citycode.isna().sum() != 0: 
res[res.city_code.isna()].city.unique()

city_codes[city_codes.city.str.contains('Berlin')]

In [ ]:
country_path = 'data/computed_data/country_codes.pickle'
with open(country_path, 'rb') as f: 
    country = pickle.load(f)
print(country.isna().sum())
country



In [ ]:
country_meta = {
    'technical': 
}


In [ ]:

#cities from pickle

cities_path = 'data/computed_data/city_codes.pickle'
with open(cities_path, 'rb') as f: 
    cities = pickle.load(f)
print(cities.isna().sum())
cities


In [ ]:
city_codes[city_codes.city.str.contains('Cairo', case=False)]

In [ ]:
cities.cc_code.str.endswith('-X').sum()

In [ ]:
cities[cities.city_code.isna()].sort_values('population', ascending=False).head(20)

In [ ]:
#cities[cities.city_code.str.contains('CN-WEI', case=False)]
cities[cities.cc_code == 'GB-STK']

In [ ]:
cities.sort_values('population', ascending=False).head(30)

In [ ]:
from datamanager import processing

numdata = pd.DataFrame(columns=['location_name', 'location_type'])


for idx, row in rawdatainfo.iterrows():
    print(f'Processing file {filename}')

    
    #
    cleaner = globals()[f'clean_{row.dataset}']

    
    # read file
    file_raw = pd.read_csv()
    
    # apply cleaning
    file = cleaner(file_raw)
    
    # join 
    numdata.merge(how='outer')
    print(idx, row)
    #print(datafile.name[idx])

## country level 

In [ ]:
country_num = country.copy()
country_num

In [ ]:
age_file = 'data/raw_data/dimensions/age_group.csv'

age = pd.read_csv(age_file)

age.rename(
    columns={
        'Country': 'country',
        'Under 5': 'babies',
        'Aged 5-14': 'children',
        'Aged 15-24': 'adolescence',
        'Aged 25-64': 'adults',
        'Over 65': 'elderly'},
    inplace=True)

age




In [ ]:
age.country_code.nunique()

In [ ]:
indices_file = 'data/raw_data/dimensions/Index.csv'
indices = pd.read_csv(indices_file)
indices

In [ ]:
wdi_file =  'data/raw_data/dimensions/WDICountry.csv'

wdi = pd.read_csv(wdi_file)
# basic cleaning

wdi.rename(columns={'2-alpha code': 'country_code', 'Currency Unit': 'currency', 'Region': 'region', 'Income Group': 'income'}, inplace=True)

print('Income: \n', wdi['income'].value_counts())

print('\nRegion: \n',wdi['region'].value_counts())

print('\nCurrency: \n',wdi['currency'].value_counts())

wdi

In [ ]:
wdi

In [ ]:
whr_file = 'data/raw_data/dimensions/whr.csv'
whr = pd.read_csv(whr_file)

print(whr.describe())

whr

## city level

### cost of living

In [ ]:

col_file = 'data/raw_data/dimensions/col.csv'

col = pd.read_csv(col_file)
col

In [ ]:
weather_file = 'data/raw_data/dimensions/weather.csv'

weather = pd.read_csv(weather_file)
weather


# to binary

In [ ]:
num_data_file = 'data/computed_data/inner_join.csv'
num_data = pd.read_csv(num_data_file)
num_data

In [ ]:
def make_binary(X):
    X['upper_quart'] = [1 if x > X.quantile(.75) else 0 for x in X]
    X['lower_quart'] = [1 if x < X.quantile(.25) else 0 for x in X]

    #bin_low = x[x<x.quantile(.25)]
    #return pd.Series({'low': lower_quart, 'high': upper_quart})
    #return pd.Series({'high': upper_quart}).transpose()
    return  X



#make_binary(num_data.Generosity)
#num_data[['Social support', 'Healthy life expectancy']].apply(make_binary, axis=0)

def make_binary(X):
    upper_quart = [1 if x > X.quantile(.75) else 0 for x in X]
    lower_quart = [1 if x < X.quantile(.25) else 0 for x in X]

    #bin_low = x[x<x.quantile(.25)]
    return pd.Series({'low': lower_quart, 'high': upper_quart})
    #return pd.Series({'high': upper_quart}).transpose()
    return  X

In [ ]:
tech_cols = ['City Code', 'Country Code', 'City', 'Country']
#cols_to_binary = num_data.columns.difference(tech_cols)
cols_to_binary = num_data.columns[~num_data.columns.isin(tech_cols)]


res = num_data[tech_cols]
for col in num_data[cols_to_binary]:
    X = num_data[col]
    res_tmp = pd.DataFrame({
        f'high {col.lower()}':[True if x > X.quantile(.75) else False for x in X],
        f'low {col.lower()}':[True if x < X.quantile(.25) else False for x in X]
    })

    #upper_quart = [1 if x > X.quantile(.75) else 0 for x in X]
    #lower_quart = [1 if x < X.quantile(.25) else 0 for x in X]
    
    #res = res.merge(pd.Series({'upper': upper_quart}), left_index=True, right_index=True)
    res = res.merge(res_tmp, left_index=True, right_index=True)

res

In [ ]:
res.to_csv('data/computed_data/data_bool.csv')

In [ ]:
filters_list = ['high logged gdp per capita', 'high social support']

def apply_filters(df, attributes): 
    filters = []
    for f in attributes: 
        filters.append(df[f])

    #res[res['high logged gdp per capita']]
    return df.loc[np.all(filters, 0), ['City Code', 'Country Code']]

apply_filters(res, filters_list)

In [ ]:
num_data[num_data['Logged GDP per capita'] > num_data['Logged GDP per capita'].quantile(.75)]